 # Topic modeling
Another way to compare documents is to extract the latent topics that group words within each document, and compare those distributions.

We'll continue on the topic of fake news with another dataset that has examples of both fake and real news articles, at a much larger scale than the previous data.

1. [Load data](#Load-data)
2. [Latent Semantic Analysis](#Latent-Semantic-Analysis)
3. [Latent Dirichlet Allocation](#Latent-Dirichlet-Allocation)
4. [Visualizing](#Visualizing-topic-models)
5. [Exploration](#Exploration)

### Load data
The data was originally released on Kaggle as a challenge to categorize fake and true data. The real/fake annotations for the data are not well-documented, but seem to be based on trustworthy vs. untrustworthy sources.

In [1]:
## data = fake news challenge
import pandas as pd
import numpy as np
fake_news_article_data = pd.read_csv('data/fake_news_challenge/Fake.csv', sep=',', index_col=False)
real_news_article_data = pd.read_csv('data/fake_news_challenge/True.csv', sep=',', index_col=False)
# full_news_article_data = pd.read_csv('data/fake_news_challenge_kaggle/train.csv', sep=',', index_col=False)
# remove null articles
# get rid of duplicate articles
fake_news_article_data.drop_duplicates('text', inplace=True)
# display(fake_news_article_data.loc[:, 'text'].head(10).values)

Before we try topic modeling, we have to convert the text to a usable format (document-term matrix, like before).

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import WordPunctTokenizer
from stop_words import get_stop_words
## combine text data, keep track of fake/real news indices
combined_news_text = fake_news_article_data.loc[:, 'text'].append(real_news_article_data.loc[:, 'text'])
fake_news_text_indices = list(range(fake_news_article_data.shape[0]))
real_news_text_indices = list(range(fake_news_article_data.shape[0], combined_news_text.shape[0]))
## convert text to DTM
en_stops = get_stop_words('en')
tokenizer = WordPunctTokenizer()
cv = CountVectorizer(min_df=0.001, max_df=0.75, lowercase=True, 
                     ngram_range=(1,1), stop_words=en_stops, tokenizer=tokenizer.tokenize)
combined_news_text_dtm = cv.fit_transform(combined_news_text)
print(combined_news_text_dtm.shape)

/home/ianbstew/miniconda3/envs/CORE_tutorial/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'aren', 'can', 'couldn', 'd', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'm', 'mustn', 're', 's', 'shan', 'shouldn', 't', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(38872, 13559)


### Latent Semantic Analysis

For our first method, let's try Latent Semantic Analysis, which is a form of dimensionality reduction.

In [3]:
## LSA
from sklearn.decomposition import TruncatedSVD
num_topics = 10
num_iter = 10
lsa_model = TruncatedSVD(n_components=num_topics, n_iter=num_iter, random_state=123)
combined_news_text_lsa_topics = lsa_model.fit_transform(combined_news_text_dtm)
print(combined_news_text_lsa_topics.shape)

(38872, 10)


The LSA process outputs continuous values [-inf, +inf] which we need to convert to probabilities [0,1]. We can use the softmax function along each dimension to convert the topic-document matrix to probabilities:

$$\text{softmax}(x_{i}) = \frac{e^{x_{i}}}{\sum_{j}^{K}e^{x_{j}}}$$

where $x$ is one of $K$ topic dimensions.

In [4]:
from sklearn.utils.extmath import softmax
from sklearn.preprocessing import StandardScaler
import numpy as np
# convert per-column scores to a normal distribution (0,1)
scaler = StandardScaler()
combined_news_text_lsa_topic_scores = scaler.fit_transform(combined_news_text_lsa_topics)
# soft-max per-column
combined_news_text_lsa_topic_probs = softmax(combined_news_text_lsa_topic_scores.T).T
# normalize per-row so that probabilities sum to 1
combined_news_text_lsa_topic_probs = combined_news_text_lsa_topic_probs / combined_news_text_lsa_topic_probs.sum(axis=1).reshape(-1,1)

What is the expected probability of a document being assigned to a topic?

In [5]:
combined_news_text_lsa_expected_topics = pd.Series(combined_news_text_lsa_topic_probs.mean(axis=0))
print(f'expected probability of topics = \n{combined_news_text_lsa_expected_topics}')

expected probability of topics = 
0    0.542500
1    0.000006
2    0.000006
3    0.000006
4    0.315447
5    0.000011
6    0.000009
7    0.133102
8    0.000013
9    0.008900
dtype: float64


It looks like the data is "dominated" by 3 topics with high probability.

To figure out what "topics" the model learned, let's look at the news articles with the highest probability for each topic.

We'll take the arg-max along each topic and print the text for the corresponding articles.
We'll look at the most likely topics (0, 4, 7) as a first pass.

In [6]:
def show_articles_with_highest_prob_per_topic(doc_topic_probs, doc_text, num_topics):
    topic_ids = list(range(num_topics))
    top_articles_per_topic = 10
    text_sample_len = 200
    for topic_id_i in topic_ids:
        print(f'processing topic {topic_id_i}')
        # get indices for articles with highest topic probability
        top_article_indices_i = np.argsort(doc_topic_probs[:, topic_id_i])[-top_articles_per_topic:]
        top_article_indices_i = list(reversed(top_article_indices_i))
        for index_j in top_article_indices_i:
            topic_prob_i_j = doc_topic_probs[index_j, topic_id_i]
            print(f'\tarticle {index_j} has P(topic)={topic_prob_i_j} with text = {doc_text.iloc[index_j][:text_sample_len]}')

In [7]:
show_articles_with_highest_prob_per_topic(combined_news_text_lsa_topic_probs, combined_news_text, num_topics)

processing topic 0
	article 9196 has P(topic)=0.9999984849945176 with text = With mainstream media and establishment politicians stacked against him from the moment he announced his run for the presidency, Donald J. Trump has been in an ongoing pitched battle to communicate hi
	article 17381 has P(topic)=0.9999967339861654 with text = Shawn Helton   21st Century WireGOP presidential frontrunner Donald Trump is a populist candidate among a bevy of warhawk rivals  yet many still wonder how the real estate mogul has marched virtually 
	article 12986 has P(topic)=0.999975002269979 with text = This is a must read for anyone who s undecided or plans on voting for a third party candidate It covers all the bases and cements for you the duty as an American to do what s best for our nation. If y
	article 16773 has P(topic)=0.9999597090174529 with text =  By ramping up US troop levels in Afghanistan, Trump is alienating many supporters. (Photo: DoD/USAF Tech Sgt Brigitte N Brantley. Source: Wikic

In [8]:
topic_ids = list(range(num_topics))
top_articles_per_topic = 10
for topic_id_i in topic_ids:
    print(f'processing topic {topic_id_i}')
    # get indices for articles with highest topic probability
    top_article_indices_i = np.argsort(combined_news_text_lsa_topic_probs[:, topic_id_i])[-top_articles_per_topic:]
    top_article_indices_i = list(reversed(top_article_indices_i))
    for index_j in top_article_indices_i:
        topic_prob_i_j = combined_news_text_lsa_topic_probs[index_j, topic_id_i]
        print(f'\tarticle {index_j} has P(topic)={topic_prob_i_j} with text = {combined_news_text.iloc[index_j][:200]}')

processing topic 0
	article 9196 has P(topic)=0.9999984849945176 with text = With mainstream media and establishment politicians stacked against him from the moment he announced his run for the presidency, Donald J. Trump has been in an ongoing pitched battle to communicate hi
	article 17381 has P(topic)=0.9999967339861654 with text = Shawn Helton   21st Century WireGOP presidential frontrunner Donald Trump is a populist candidate among a bevy of warhawk rivals  yet many still wonder how the real estate mogul has marched virtually 
	article 12986 has P(topic)=0.999975002269979 with text = This is a must read for anyone who s undecided or plans on voting for a third party candidate It covers all the bases and cements for you the duty as an American to do what s best for our nation. If y
	article 16773 has P(topic)=0.9999597090174529 with text =  By ramping up US troop levels in Afghanistan, Trump is alienating many supporters. (Photo: DoD/USAF Tech Sgt Brigitte N Brantley. Source: Wikic

Looking at the article text qualitatively, we observe the following:

- Topic 0 includes major executive-branch issues such as U.S. president Trump's campaign and action in office.
- Topic 4 includes more subjective claims (`anti-American`, `whine`) and more extreme issues (`conspiracy`, `chaos`, `violence`).
- Topic 7 includes discussion of the 2016 election, particularly related to Clinton (`investigation`, `email`, `classified`).

Which topics are more prevalent in fake news versus real news?

In [9]:
fake_news_text_lsa_topic_probs = combined_news_text_lsa_topic_probs[fake_news_text_indices, :]
real_news_text_lsa_topic_probs = combined_news_text_lsa_topic_probs[real_news_text_indices, :]
fake_news_text_lsa_expected_topics = pd.Series(fake_news_text_lsa_topic_probs.mean(axis=0))
real_news_text_lsa_expected_topics = pd.Series(real_news_text_lsa_topic_probs.mean(axis=0))
print(f'expected probability of topics for fake news = \n{fake_news_text_lsa_expected_topics}')
print(f'expected probability of topics for real news = \n{real_news_text_lsa_expected_topics}')

expected probability of topics for fake news = 
0    0.502965
1    0.000014
2    0.000014
3    0.000014
4    0.364562
5    0.000024
6    0.000019
7    0.125921
8    0.000029
9    0.006437
dtype: float64
expected probability of topics for real news = 
0    5.747211e-01
1    4.791793e-50
2    1.157458e-21
3    1.558271e-08
4    2.754175e-01
5    4.933393e-16
6    4.747155e-12
7    1.389541e-01
8    6.871054e-07
9    1.090660e-02
dtype: float64


It looks like real news discusses topic 0 (possible criticism of Trump?) slightly more than fake news, while fake news discusses discusses topic 4 (conspiracy theories?) slightly more than real news.

While this is a useful first pass on the data, it doesn't help us identify which words or phrases may differentiate fake news from real news. 

We'll move onto a more complicated method (Latent Dirichlet Allocation) that identifies latent topics from which words are "generated." 
This will help us pull out specific words that characterize the topics.

### Latent Dirichlet Allocation

In [13]:
## LDA
# get text tokens first using the CountVectorizer from earlier
combined_news_text_dtm_tokens = cv.inverse_transform(combined_news_text_dtm)
from gensim.corpora import Dictionary
lda_dict = Dictionary(combined_news_text_dtm_tokens)
combined_news_text_corpus = list(map(lambda x: lda_dict.doc2bow(x), combined_news_text_dtm_tokens))
# train model
from gensim.models import LdaModel
num_topics = 10
iterations = 50
lda_model = LdaModel(corpus=combined_news_text_corpus, num_topics=10, iterations=iterations, random_state=123)

Like before, let's look at the distribution of topics over all documents and get a sense of the articles that correspond to each topic.

In [15]:
def compute_lda_topic_probs(text_doc, model):
    doc_topics = model.get_document_topics(text_doc, minimum_probability=0.)
    # convert to probability array
    doc_topic_ids, doc_topic_probs = zip(*doc_topics)
    return doc_topic_probs
combined_news_text_lda_topic_probs = np.array(list(map(lambda x: compute_lda_topic_probs(x, lda_model), combined_news_text_corpus)))
combined_news_text_lda_topic_expected_prob = combined_news_text_lda_topic_probs.mean(axis=0)
print(f'expected value of LDA topics =\n{combined_news_text_lda_topic_expected_prob}')

expected value of LDA topics =
[0.1350052  0.04586368 0.09425619 0.06259541 0.07729013 0.15083285
 0.16970329 0.09634831 0.10720032 0.06090749]


In contrast to the SVD model, here we see a more even distribution of topics. Let's see which articles were more strongly associated with each topic.

In [16]:
show_articles_with_highest_prob_per_topic(combined_news_text_lda_topic_probs, combined_news_text, num_topics)

processing topic 0
	article 30135 has P(topic)=0.9958608150482178 with text = UNITED NATIONS (Reuters) - Britain, France, Germany, Sweden and Italy called on the United States on Friday to put forward detailed proposals for peace between Israel and the Palestinians and describe
	article 36738 has P(topic)=0.9928897619247437 with text = BEIJING (Reuters) - Japan talking only about sanctions on North Korea rather than dialogue will be seen as going against United Nations resolutions, Chinese Foreign Minister Wang Yi told his Japanese 
	article 38026 has P(topic)=0.9920267462730408 with text = MEXICO CITY (Reuters) - North Korea s ambassador to Mexico on Friday said its tensions with the United States were not Mexico City s business after President Enrique Pena Nieto ordered that he leave t
	article 32409 has P(topic)=0.9918334484100342 with text = MANILA (Reuters) - U.S. President Donald Trump raised North Korea s missile tests during talks on Monday with the prime ministers of Japan and

Restricting ourselves to the top 5 most frequent topics in the data based on the probabilities above (topics 0, 3, 5, 6, 8), we see the following trends:

- Topic 0 includes international politics.
- Topic 3 includes immigration, possibly anti-Muslim sentiment.
- Topic 5 includes tax debates and financial negotiations in the U.S. legislative branch.
- Topic 6 includes opinion pieces concerning the actions of politicians (especially Donald Trump).
- Topic 8 includes election investigation cases.

Let's also compare the distribution of topics in each text category.

In [17]:
fake_news_text_lda_topic_probs = combined_news_text_lda_topic_probs[fake_news_text_indices, :]
real_news_text_lda_topic_probs = combined_news_text_lda_topic_probs[real_news_text_indices, :]
fake_news_text_lda_expected_topics = pd.Series(fake_news_text_lda_topic_probs.mean(axis=0))
real_news_text_lda_expected_topics = pd.Series(real_news_text_lda_topic_probs.mean(axis=0))
print(f'expected probability of topics for fake news = \n{fake_news_text_lda_expected_topics}')
print(f'expected probability of topics for real news = \n{real_news_text_lda_expected_topics}')

expected probability of topics for fake news = 
0    0.038589
1    0.013171
2    0.117071
3    0.054601
4    0.136850
5    0.058710
6    0.351804
7    0.070792
8    0.078739
9    0.079672
dtype: float32
expected probability of topics for real news = 
0    0.213585
1    0.072508
2    0.075662
3    0.069111
4    0.028748
5    0.225914
6    0.021285
7    0.117175
8    0.130397
9    0.045614
dtype: float32


Real news articles tend to have more representation for topics 0, 3, and 5 (more "standard" news topics concerning every day affairs), while fake news articles have more representation for topics 6 and 8 (more Trump-centric and controversial content).

Now that we've established the high-level differences in topics between fake news and real news, let's look at the individual words that make up the topics.

Specifically, we're going to compute the probability of observing a word given a topic, using the parameters learned by the LDA model.

In [18]:
def show_top_words_all_topics(model, model_dict, num_topics, words_per_topic):
    topic_ids = list(range(num_topics))
    for topic_i in topic_ids:
        topic_word_id_scores_i = model.get_topic_terms(topic_i, topn=words_per_topic)
        topic_word_ids_i, topic_word_scores_i = zip(*topic_word_id_scores_i)
        # convert word ID to words
        topic_words_i = list(map(model_dict.get, topic_word_ids_i))
        print(f'topic {topic_i} has top words: \n\t{", ".join(topic_words_i)}')

In [19]:
words_per_topic = 20
show_top_words_all_topics(lda_model, lda_dict, num_topics, words_per_topic)

topic 0 has top words: 
	reuters, minister, (, united, president, will, states, prime, told, government, foreign, country, also, european, state, countries, u, leader, nations, two
topic 1 has top words: 
	(, military, forces, reuters, al, army, killed, militants, air, islamic, syria, u, turkey, state, government, group, security, fighters, fighting, near
topic 2 has top words: 
	people, police, year, (, one, old, city, two, 000, many, killed, also, home, three, violence, around, local, authorities, told, last
topic 3 has top words: 
	court, reuters, (, rights, law, case, ruling, police, authorities, justice, people, charges, supreme, arrested, legal, government, arrest, trial, detained, last
topic 4 has top words: 
	social, media, political, people, one, (, :, anti, world, t, like, right, many, national, first, country, university, time, also, speech
topic 5 has top words: 
	reuters, (, ..., president, will, u, “, ’, ,”, percent, election, washington, house, vote, donald, new, trump, 

- Topic 0 includes more international relations words (`states`, `foreign`, `nations`).
- Topic 3 includes more legal words (`law`, `arrest`, `justice`).
- Topic 5 includes more executive-branch words (`president`, `washington`).
- Topic 6 includes more president-specific words, and more subjective (?) words (`think`, `know`, `just`).
- Topic 8 includes words related to Russian affairs and the election investigation (`russia`, `investigation`).

What happens if we train separate topic models on real news and fake news? This could help highlight groups of words that are specific only to fake news or to real news, which may be "washed out" with the combined topic model.

In [21]:
num_topics = 10
iterations = 100
# train fake news model
def train_lda_model_from_corpus(text_corpus, num_topics, iterations):
    lda_model = LdaModel(text_corpus, num_topics=num_topics, iterations=iterations, random_state=123)
    ## add dictionary
    return lda_model
# fake_news_text_dtm_tokens = list(map(lambda x: combined_news_text_dtm_tokens[x], fake_news_text_indices))
# real_news_text_dtm_tokens = list(map(lambda x: combined_news_text_dtm_tokens[x], real_news_text_indices))
fake_news_text_corpus = list(map(lambda x: combined_news_text_corpus[x], fake_news_text_indices))
real_news_text_corpus = list(map(lambda x: combined_news_text_corpus[x], real_news_text_indices))
## train models
fake_news_lda_model = train_lda_model_from_corpus(fake_news_text_corpus, num_topics, iterations)
real_news_lda_model = train_lda_model_from_corpus(real_news_text_corpus, num_topics, iterations)

What are the top words captured per-topic from each model?

In [23]:
words_per_topic = 20
print('real news: top words per topic')
show_top_words_all_topics(real_news_lda_model, lda_dict, num_topics, words_per_topic)

real news: top words per topic
topic 0 has top words: 
	reuters, (, people, one, police, two, country, year, killed, 000, government, last, since, also, state, years, told, city, many, will
topic 1 has top words: 
	reuters, (, minister, government, told, will, u, president, prime, state, military, islamic, european, forces, turkey, al, statement, also, last, wednesday
topic 2 has top words: 
	party, vote, election, majority, (, immigration, reuters, conservative, will, also, support, opposition, new, conservatives, democrats, michel, right, government, year, votes
topic 3 has top words: 
	reuters, (, president, told, u, rodrigo, also, government, will, philippines, manila, people, wednesday, former, philippine, washington, year, reporters, trump, last
topic 4 has top words: 
	reuters, (, u, president, beirut, trump, donald, washington, will, energy, global, oil, house, climate, also, agency, “, administration, thursday, environmental
topic 5 has top words: 
	percent, $, 1, million, yea

In [24]:
print('fake news: top words per topic')
show_top_words_all_topics(fake_news_lda_model, lda_dict, num_topics, words_per_topic)

fake news: top words per topic
topic 0 has top words: 
	:, will, t, people, can, one, via, america, like, states, just, president, now, american, country, us, also, obama, republican, many
topic 1 has top words: 
	:, (, new, also, million, will, one, t, 1, 21st, just, $, percent, people, according, news, 5, ?, even, can
topic 2 has top words: 
	:, t, one, ?, also, via, news, told, just, people, can, former, now, will, (, president, even, trump, like, two
topic 3 has top words: 
	:, t, ?, !, people, like, one, just, can, trump, /, don, re, video, via, know, get, us, america, watch
topic 4 has top words: 
	);, script, connect, sexual, ;, document, id, 3, 1, function, =, net, //, return, [, cdata, 0, version, var, ))
topic 5 has top words: 
	t, domestically, :, screening, even, like, people, just, will, president, remote, americans, one, san, obamacare, health, can, defence, country, also
topic 6 has top words: 
	police, :, officers, shooting, killed, old, one, man, t, city, shot, year, s

The real news articles include discussions of international relations (topic 1, topic 7); hedging words to make claims "softer" (topic 8 `alleged`, `reported`); legal issues (topic 9); the energy industry (topic 4). In general, these are topics that I expect from a typical news paper.

The fake news articles include more subjective words (topics 1 and 3 `just`, `know`, `even`); violent/fear issues (topic 6 `police`, `killed`); American identity (?) (topic 0 `america`, `american`, `republican`; topic 5 `americans`, `domestically`); discussions of left-wing politicians, presumably negative (topic 8 `obama`; topic 9 `hillary`).

### Visualizing topic models

One last extra step if we have time: we can visualize the topics that the models have learned to understand the relationship between the topics.

Some topics may be closer together in "space" than others. 
For instance, topics that discuss different aspects of international relations. 
[The pyLDAvis package](https://github.com/bmabey/pyLDAvis) visualizes the relationship between LDA topics by projecting the topics to a shared 2-dimensional space via Principal Components Analysis. 

In [28]:
import pyLDAvis
pyLDAvis.enable_notebook()

/home/ianbstew/miniconda3/envs/CORE_tutorial/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
## need to make separate dict for each corpus because of internal errors
from gensim.corpora import Dictionary
def generate_dict_corpus_model_for_data(combined_data_tokens, data_indices, num_topics, iterations):
    data_tokens = list(map(lambda x: combined_data_tokens[x], data_indices))
    data_dict = Dictionary(data_tokens)
    data_corpus = list(map(lambda x: data_dict.doc2bow(x), data_tokens))
    lda_model = train_lda_model_from_corpus(data_corpus, num_topics, iterations)
    return data_dict, data_corpus, lda_model
num_topics = 10
iterations = 50
clean_fake_news_text_dict, clean_fake_news_text_corpus, clean_fake_news_lda_model = generate_dict_corpus_model_for_data(combined_news_text_dtm_tokens, 
                                                                                                                        fake_news_text_indices, 
                                                                                                                        num_topics, iterations)
clean_real_news_text_dict, clean_real_news_text_corpus, clean_real_news_lda_model = generate_dict_corpus_model_for_data(combined_news_text_dtm_tokens, 
                                                                                                                        real_news_text_indices, 
                                                                                                                        num_topics, iterations)

/home/ianbstew/miniconda3/envs/CORE_tutorial/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
## plot topics learned on real news
import pyLDAvis.gensim
pyLDAvis.gensim.prepare(clean_real_news_lda_model, clean_real_news_text_corpus, clean_real_news_text_dict)

/home/ianbstew/miniconda3/envs/CORE_tutorial/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.068100  0.069845       1        1  16.968294
4     -0.063204 -0.072396       2        1  11.669899
5     -0.014405  0.085393       3        1  11.277580
9      0.041946  0.033611       4        1  10.346703
7      0.007396  0.033035       5        1  10.306456
3     -0.093900 -0.102913       6        1   9.468217
6      0.159677 -0.019226       7        1   8.558801
0      0.099913 -0.001494       8        1   8.048081
2      0.024145 -0.077412       9        1   6.919701
8     -0.093467  0.051556      10        1   6.436268, topic_info=            Term         Freq        Total Category  logprob  loglift
696      russian  3182.000000  3182.000000  Default  30.0000  30.0000
1083    minister  5721.000000  5721.000000  Default  29.0000  29.0000
682       moscow  2448.000000  2448.000000  Default  28.0000  28.0000
587       russia  3403.000000  3403.000000  Default  27.0000  27.0000
1073       korea  2011.000000  2011.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
669    including   441.735879  5965.543056  Topic10  -6.2072   0.1402
121   government   455.225136  9634.508176  Topic10  -6.1771  -0.3091
279         told   447.652225  9906.091645  Topic10  -6.1939  -0.3537
216       public   420.015072  4639.311164  Topic10  -6.2576   0.3412
491         time   420.798538  5538.523458  Topic10  -6.2557   0.1659

[915 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
0          1  0.473384         $
0          2  0.010943         $
0          3  0.199797         $
0          4  0.011256         $
0          5  0.015946         $
...      ...       ...       ...
11208      2  0.719639  zimbabwe
11208      3  0.248046  zimbabwe
11208      4  0.003062  zimbabwe
11208      5  0.024498  zimbabwe
11686      8  0.993757       zor

[4255 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 6, 10, 8, 4, 7, 1, 3, 9])

The main dimensions for real news seem to be "domestic" vs. "international" (PC1) and "facts" vs. "subjective/controversy" (PC2).

In [47]:
## plot topics learned on fake news
import pyLDAvis.gensim
pyLDAvis.gensim.prepare(clean_fake_news_lda_model, clean_fake_news_text_corpus, clean_fake_news_text_dict)

/home/ianbstew/miniconda3/envs/CORE_tutorial/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.006814  0.087328       1        1  19.286125
0      0.040644 -0.003219       2        1  16.493891
7     -0.000123 -0.085841       3        1  12.595340
2      0.023737  0.012651       4        1  11.164360
8      0.016399 -0.043492       5        1  10.821982
9      0.006878  0.070831       6        1   7.411590
1      0.068106 -0.044171       7        1   6.987671
6     -0.109584  0.059483       8        1   6.503181
4     -0.116631 -0.068702       9        1   6.080532
5      0.063760  0.015133      10        1   2.655327, topic_info=        Term         Freq        Total Category  logprob  loglift
1         (@  3577.000000  3577.000000  Default  30.0000  30.0000
141    trump  8045.000000  8045.000000  Default  29.0000  29.0000
145  twitter  3782.000000  3782.000000  Default  28.0000  28.0000
117      pic  2425.000000  2425.000000  Default  27.0000  27.0000
39       com  3974.000000  3974.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
423    think   175.515759  4232.615010  Topic10  -6.0761   0.4458
21      also   186.235102  6349.400816  Topic10  -6.0168   0.0995
148     want   170.632816  3701.709090  Topic10  -6.1043   0.5516
16         ?   187.663853  9096.331538  Topic10  -6.0091  -0.2524
456     back   167.711450  4464.948867  Topic10  -6.1216   0.3468

[940 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.423620    !
0         2  0.055497    !
0         3  0.031189    !
0         4  0.067504    !
0         5  0.071018    !
...     ...       ...  ...
4483      6  0.424140    |
4483      7  0.012475    |
4483      8  0.042102    |
4483      9  0.082645    |
4483     10  0.003119    |

[4225 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 8, 3, 9, 10, 2, 7, 5, 6])

The main dimensions for fake news seem to be "news" vs. "opinion" (PC1) and "government" vs. "people" (PC2).

### Exploration
Now it's time for you to keep exploring what topic models can tell us about real and fake news.

Some ideas:
- We used word frequency to represent words when training the topic models, but you can try other metrics such as TF-IDF, which we saw before can up-weight rarer words. What happens if you re-train the topic model using another form of word frequency?
- You can change the number of topics learned by the model to include more or less detail that may reveal different "levels" of granularity. You may want to try using "coherence" as a metric to determine the number of topics that maximizes the similarity among words within the same topic. What broad or fine-grained differences can you find that differentiate real and fake news? 
- One way of reducing "overlap" among words within topics is to **stem** each word and convert it to a base form that is shared among different versions of the word (e.g. `dog` and `dogs` stemmed to `dog`). What happens if you stem the text before training the topic model?